# Marginal Effects in Censored and Selection Models

**Series:** Marginal Effects Tutorial — Notebook 4 of 6  
**Level:** Advanced  
**Estimated time:** 75–90 minutes  
**Prerequisites:** Notebook 01 (ME Fundamentals), familiarity with Tobit/Heckman theory

---

## Motivation

Models where the dependent variable is **limited** — either censored (Tobit) or only observed
for a selected subsample (Heckman) — require careful treatment of marginal effects.
There are multiple valid definitions, each answering a **different economic question**.

### Three questions a researcher may ask

1. "How does x affect the **latent** outcome (e.g., *desired* labor supply)?"
2. "How does x affect the **observed** outcome (including zeros)?"
3. "How does x affect the outcome **conditional on being non-zero**?"

Each question leads to a different formula. This notebook covers all three for the Tobit model,
the McDonald-Moffitt decomposition, and the direct/indirect decomposition for Heckman.

---

## Table of Contents

1. [Three Types of Marginal Effects in Tobit](#section1)
2. [McDonald-Moffitt Decomposition: Extensive vs Intensive Margin](#section2)
3. [Heckman Selection Model — Direct and Indirect Effects](#section3)
4. [When to Use Tobit vs Heckman?](#section4)
5. [Key Takeaways](#takeaways)

In [ ]:
import sys
sys.path.insert(0, '/home/guhaase/projetos/panelbox')
sys.path.insert(0, '..')

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from scipy.stats import norm

# PanelBox imports
from panelbox.models.censored.tobit import PooledTobit
from panelbox.marginal_effects.censored_me import (
    compute_tobit_ame,
    compute_tobit_mem,
)

# Utility imports
from utils.data_loaders import load_dataset
from utils.me_helpers import plot_forest, format_me_table

plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.float_format', '{:.4f}'.format)
print("Setup complete.")

---

<a id='section1'></a>
## Section 1: Three Types of Marginal Effects in Tobit

### Tobit latent-variable setup

$$Y^* = X\beta + \varepsilon, \quad \varepsilon \sim N(0, \sigma^2)$$
$$Y = \max(Y^*, 0) \quad \text{[left-censored at zero]}$$

Let $z = X\beta / \sigma$. Then:

| Type | Formula | Interpretation | Use when |
|------|---------|----------------|----------|
| **Type 1** — Latent ME | $\partial E[Y^*\|X]/\partial x_k = \beta_k$ | Effect if censoring were lifted | Structural/theoretical interest |
| **Type 2** — Unconditional ME | $\partial E[Y\|X]/\partial x_k = \beta_k \cdot \Phi(z)$ | Population-average effect incl. zeros | Policy / welfare analysis |
| **Type 3** — Conditional ME | $\partial E[Y\|Y>0,X]/\partial x_k = \beta_k \cdot [1 - \lambda(z)(z+\lambda(z))]$ | Effect for participants only | Intensive-margin analysis |

where $\Phi$ is the standard normal CDF and $\lambda(z) = \phi(z)/\Phi(z)$ is the inverse Mills ratio.

> **Note:** $\Phi(z) \leq 1$, so the unconditional ME is always **smaller in absolute value** than the latent ME.

In [ ]:
# Load Mroz hours dataset
df = load_dataset('mroz_hours')
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nHours worked distribution:")
print(f"  % zero (not working): {(df['hours'] == 0).mean():.1%}")
print(f"  Mean hours (all):     {df['hours'].mean():.1f}")
print(f"  Mean hours (workers): {df.loc[df['hours'] > 0, 'hours'].mean():.1f}")

# Visualize censoring
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(df['hours'], bins=40, color='steelblue', alpha=0.75, edgecolor='white')
axes[0].set_xlabel("Hours Worked per Year")
axes[0].set_ylabel("Count")
axes[0].set_title("Full Distribution (including zeros)")

axes[1].hist(df.loc[df['hours'] > 0, 'hours'], bins=35,
             color='tomato', alpha=0.75, edgecolor='white')
axes[1].set_xlabel("Hours Worked per Year")
axes[1].set_title("Conditional on Working (hours > 0)")

plt.suptitle("Mroz (1987) — Hours Worked: Evidence of Left Censoring at Zero",
             fontsize=12, y=1.01)
plt.tight_layout()

import os
os.makedirs('../outputs/plots', exist_ok=True)
plt.savefig('../outputs/plots/04_hours_distribution.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"\nPlot saved.")

In [ ]:
# Estimate Pooled Tobit using arrays directly
covariates = ['educ', 'age', 'kidslt6', 'kidsge6', 'nwifeinc']

y = df['hours'].values
X = np.column_stack([np.ones(len(df))] + [df[v].values for v in covariates])
exog_names = ['const'] + covariates

print("Fitting Pooled Tobit (left-censored at 0) ...")
print(f"  Observations: {len(y)}")
print(f"  Censored (hours=0): {(y == 0).sum()} ({(y == 0).mean():.1%})")
print(f"  Covariates: {covariates}")
print()

try:
    tobit = PooledTobit(
        endog=y,
        exog=X,
        censoring_point=0,
        censoring_type='left'
    ).fit()

    # Set variable names manually
    tobit.exog_names = exog_names

    # Display results
    print("=" * 60)
    print("Pooled Tobit Results")
    print("=" * 60)
    print(f"{'Variable':<15} {'Coef.':<12} {'Std.Err.':<12} {'z':<8} {'P>|z|':<8}")
    print("-" * 60)

    from scipy.stats import norm as norm_dist
    for i, name in enumerate(exog_names):
        coef = tobit.beta[i]
        se_val = tobit.bse[i] if hasattr(tobit, 'bse') and not np.isnan(tobit.bse[i]) else np.nan
        if not np.isnan(se_val) and se_val > 0:
            z_stat = coef / se_val
            p_val = 2 * norm_dist.sf(abs(z_stat))
            print(f"  {name:<13} {coef:>10.4f} {se_val:>10.4f} {z_stat:>7.2f} {p_val:>7.4f}")
        else:
            print(f"  {name:<13} {coef:>10.4f} {'n/a':>10} {'n/a':>7} {'n/a':>7}")

    print("-" * 60)
    print(f"  sigma (sigma_eps): {tobit.sigma:.4f}")
    print(f"  Log-likelihood:    {tobit.llf:.3f}")
    print(f"  Converged:         {tobit.converged}")
    print("=" * 60)
    print()
    print("Interpretation:")
    print(f"  sigma = {tobit.sigma:.2f} is the std dev of the latent variable error term.")
    tobit_fitted = True

except Exception as e:
    print(f"Tobit estimation failed: {e}")
    tobit_fitted = False

In [ ]:
if tobit_fitted:
    print("=" * 60)
    print("THREE TYPES OF TOBIT MARGINAL EFFECTS (AME)")
    print("=" * 60)

    # Type 1: Effect on latent variable (= beta)
    latent_me = pd.Series(
        {name: tobit.beta[i] for i, name in enumerate(exog_names)},
        name='Latent ME (=beta)'
    )

    # Type 2: Unconditional ME = beta * Phi(z)
    try:
        ame_uncond = compute_tobit_ame(tobit, which='unconditional', varlist=covariates)
        print("\n--- Type 2: Unconditional ME (includes zeros) ---")
        print(ame_uncond.summary())
        ame_uncond_ok = True
    except Exception as e:
        print(f"Unconditional ME failed: {e}")
        ame_uncond_ok = False

    # Type 3: Conditional ME = beta * [1 - lambda(z)(z + lambda(z))]
    try:
        ame_cond = compute_tobit_ame(tobit, which='conditional', varlist=covariates)
        print("\n--- Type 3: Conditional ME (given hours > 0) ---")
        print(ame_cond.summary())
        ame_cond_ok = True
    except Exception as e:
        print(f"Conditional ME failed: {e}")
        ame_cond_ok = False

    # Probability ME = (beta/sigma) * phi(z)
    try:
        ame_prob = compute_tobit_ame(tobit, which='probability', varlist=covariates)
        print("\n--- Effect on P(working) [Probability ME] ---")
        print(ame_prob.summary())
        ame_prob_ok = True
    except Exception as e:
        print(f"Probability ME failed: {e}")
        ame_prob_ok = False

else:
    print("Skipping ME computation — Tobit did not converge.")
    ame_uncond_ok = ame_cond_ok = ame_prob_ok = False

In [ ]:
if tobit_fitted and ame_uncond_ok and ame_cond_ok and ame_prob_ok:
    # Build comparison table
    comparison = pd.DataFrame({
        'Latent (beta)':  [tobit.beta[exog_names.index(v)] for v in covariates],
        'Unconditional':  [ame_uncond.marginal_effects[v] for v in covariates],
        'Conditional':    [ame_cond.marginal_effects[v]   for v in covariates],
        'P(working)':     [ame_prob.marginal_effects[v]   for v in covariates],
    }, index=covariates)

    print("=" * 70)
    print("ALL THREE TOBIT ME TYPES — SIDE-BY-SIDE COMPARISON")
    print("=" * 70)
    print(comparison.round(4).to_string())

    print("\nKey insights:")
    print("  - |Unconditional| < |Latent|: because Phi(z) <= 1")
    print("  - |Conditional| < |Latent|:  because [1-lambda(z+lambda)] <= 1")
    print("  - P(working) ME gives a probability, not hours")

    # Quick forest plot — unconditional AME
    fig = plot_forest(
        ame_uncond,
        title="Tobit Unconditional AME — Effect on Hours Worked",
        figsize=(8, 5)
    )
    plt.savefig('../outputs/plots/04_tobit_ame_unconditional.png', dpi=150, bbox_inches='tight')
    plt.show()

elif tobit_fitted:
    # Manual fallback comparison
    print("Computing ME manually for comparison...")
    z_i = (X @ tobit.beta - 0) / tobit.sigma
    Phi_z = norm.cdf(z_i)
    phi_z = norm.pdf(z_i)
    lambda_z = phi_z / np.where(Phi_z > 1e-10, Phi_z, 1e-10)

    rows = []
    for v in covariates:
        idx = exog_names.index(v)
        bk = tobit.beta[idx]
        rows.append({
            'Variable': v,
            'Latent (beta)': bk,
            'Unconditional': np.mean(bk * Phi_z),
            'Conditional':   np.mean(bk * (1 - lambda_z * (z_i + lambda_z))),
            'P(working)':    np.mean((bk / tobit.sigma) * phi_z),
        })
    comparison = pd.DataFrame(rows).set_index('Variable')
    print(comparison.round(4).to_string())

else:
    print("Skipping comparison — model not fitted.")

---

<a id='section2'></a>
## Section 2: McDonald-Moffitt Decomposition

**McDonald & Moffitt (1980)** showed that the **unconditional ME** (Type 2) can be decomposed into two economically meaningful components:

$$
\underbrace{\frac{\partial E[Y|X]}{\partial x_k}}_{\text{Unconditional ME}}
=
\underbrace{\frac{\partial P(Y>0|X)}{\partial x_k} \cdot E[Y|Y>0, X]}_{\text{Extensive Margin}}
+
\underbrace{P(Y>0|X) \cdot \frac{\partial E[Y|Y>0,X]}{\partial x_k}}_{\text{Intensive Margin}}
$$

**Extensive margin:** effect of $x_k$ on the **probability of participation** times the
conditional mean hours  
**Intensive margin:** probability of participation times the effect on **hours given working**

These two always sum to the unconditional ME.

### Economic interpretation (labor supply)

- **Education** likely increases both LFP probability (extensive) and conditional hours (intensive)
- **Young children** mainly reduce LFP (extensive) more than reducing hours of those who already work (intensive)

In [ ]:
if tobit_fitted:
    # McDonald-Moffitt decomposition at sample means
    # Compute z at every observation, then average
    xb = X @ tobit.beta
    z_all = (xb - 0) / tobit.sigma  # censoring point = 0

    phi_z_all = norm.pdf(z_all)
    Phi_z_all = norm.cdf(z_all)
    # Safe inverse Mills ratio
    lambda_z_all = np.where(
        Phi_z_all > 1e-10,
        phi_z_all / Phi_z_all,
        -z_all  # asymptotic approximation for extreme negatives
    )

    # Conditional mean E[Y|Y>0, X] at each observation
    cond_mean_all = xb + tobit.sigma * lambda_z_all

    # Decomposition for each covariate
    rows = []
    for v in covariates:
        idx = exog_names.index(v)
        bk = tobit.beta[idx]
        sigma = tobit.sigma

        # Extensive margin: (beta_k/sigma) * phi(z) * E[Y|Y>0, X]
        extensive_i = (bk / sigma) * phi_z_all * cond_mean_all

        # Intensive margin: Phi(z) * beta_k * [1 - lambda(z)(z + lambda(z))]
        intensive_i = Phi_z_all * bk * (1 - lambda_z_all * (z_all + lambda_z_all))

        # AME versions (average over sample)
        ext_ame = np.mean(extensive_i)
        int_ame = np.mean(intensive_i)
        total_ame = ext_ame + int_ame

        # Verify against compute_tobit_ame if available
        if ame_uncond_ok:
            verify = ame_uncond.marginal_effects[v]
        else:
            verify = np.nan

        rows.append({
            'Variable': v,
            'Extensive Margin': ext_ame,
            'Intensive Margin': int_ame,
            'Total (McDonald-Moffitt)': total_ame,
            'Unconditional ME (verify)': verify,
        })

    decomp = pd.DataFrame(rows).set_index('Variable')
    print("=" * 75)
    print("McDonald-Moffitt Decomposition (AME version)")
    print("=" * 75)
    print(decomp.round(4).to_string())
    print()
    print("Check: Total should match Unconditional ME (within numerical precision).")

    # Store for later cells
    extensive_series = pd.Series({r['Variable']: r['Extensive Margin'] for r in rows})
    intensive_series = pd.Series({r['Variable']: r['Intensive Margin'] for r in rows})
    total_series     = pd.Series({r['Variable']: r['Total (McDonald-Moffitt)'] for r in rows})

    mcdonald_ok = True
else:
    print("Skipping McDonald-Moffitt — Tobit not fitted.")
    mcdonald_ok = False

In [ ]:
if mcdonald_ok:
    # Only plot variables with meaningful total ME
    vars_plot = [v for v in covariates if abs(float(total_series[v])) > 0.1]
    if not vars_plot:
        vars_plot = covariates  # fallback: show all

    x_pos = np.arange(len(vars_plot))
    bars_ext = [float(extensive_series[v]) for v in vars_plot]
    bars_int = [float(intensive_series[v]) for v in vars_plot]

    fig, ax = plt.subplots(figsize=(9, 5))

    # Stacked bars — handle negative values
    pos_ext = np.maximum(bars_ext, 0)
    neg_ext = np.minimum(bars_ext, 0)
    pos_int = np.maximum(bars_int, 0)
    neg_int = np.minimum(bars_int, 0)

    ax.bar(x_pos, bars_ext, label='Extensive Margin', color='steelblue', alpha=0.85)
    ax.bar(x_pos, bars_int, bottom=bars_ext, label='Intensive Margin', color='tomato', alpha=0.85)

    # Add total labels
    for i, v in enumerate(vars_plot):
        total_val = float(total_series[v])
        ax.text(i, total_val + (5 if total_val >= 0 else -15),
                f'{total_val:.1f}', ha='center', va='bottom', fontsize=8, fontweight='bold')

    ax.axhline(0, color='black', lw=0.8)
    ax.set_xticks(x_pos)
    ax.set_xticklabels(vars_plot, rotation=15, ha='right')
    ax.set_ylabel("Marginal Effect on Hours Worked")
    ax.set_title(
        "McDonald-Moffitt Decomposition: Extensive vs Intensive Margin\n"
        "Tobit Model — Women's Labor Supply (Mroz 1987)"
    )
    ax.legend()
    plt.tight_layout()
    plt.savefig('../outputs/plots/04_mcdonald_moffitt_decomposition.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Plot saved.")
else:
    print("Skipping plot — decomposition not available.")

---

<a id='section3'></a>
## Section 3: Heckman Selection Model — Direct and Indirect Effects

### The sample selection problem

Heckman (1979) two-equation model:

$$
\text{Selection: } D = \mathbf{1}[Z\gamma + u > 0] \quad (\text{observe }Y\text{ only if }D=1)
$$
$$
\text{Outcome: } Y = X\beta + \varepsilon \quad (\text{only if }D=1)
$$
$$
\text{Corr}(u,\varepsilon) = \rho \neq 0 \quad (\text{selection endogeneity})
$$

Two-stage correction using Inverse Mills Ratio (IMR):

$$
E[Y|D=1, X] = X\beta + \underbrace{\rho\sigma_\varepsilon}_{\delta} \cdot \lambda(Z\gamma)
$$

where $\lambda(Z\gamma) = \phi(Z\gamma)/\Phi(Z\gamma)$ is the IMR.

### Decomposition of ME

For $x_k$ that appears in both $X$ and $Z$:

$$
\frac{\partial E[Y|D=1,X]}{\partial x_k}
= \underbrace{\beta_k}_{\text{Direct}}
+ \underbrace{\delta \cdot \frac{\partial \lambda(Z\gamma)}{\partial x_k}}_{\text{Indirect (selection)}}
$$

### Exclusion restriction

For identification, $Z$ must contain at least one variable **not in** $X$.  
Example: young children affect LFP (selection) but not the wage conditional on working.

In [ ]:
# Attempt Heckman estimation — wrapped in try/except
heckman_fitted = False
heckman_result = None

try:
    from panelbox.models.selection.heckman import PanelHeckman

    df_heck = load_dataset('mroz')
    print(f"Mroz dataset shape: {df_heck.shape}")
    print(f"Columns: {list(df_heck.columns)}")

    # Selection equation: LFP ~ educ + age + kidslt6 + kidsge6 + nwifeinc
    # Outcome equation:   log(wage) ~ educ + exper + expersq
    # Exclusion variables: kidslt6, kidsge6, nwifeinc (selection only)

    sel_covs = ['educ', 'age', 'kidslt6', 'kidsge6', 'nwifeinc']
    out_covs  = ['educ', 'exper', 'expersq']

    D = df_heck['inlf'].values  # selection indicator

    # Outcome variable: log wage (only observed for D=1)
    wage_col = 'wage'
    df_heck['log_wage'] = np.where(df_heck[wage_col] > 0,
                                    np.log(df_heck[wage_col].clip(lower=0.01)),
                                    np.nan)
    y_out = df_heck['log_wage'].values

    # Build design matrices
    X_sel = np.column_stack([np.ones(len(df_heck))] + [df_heck[v].values for v in sel_covs])
    X_out = np.column_stack([np.ones(len(df_heck))] + [df_heck[v].values for v in out_covs])

    heckman = PanelHeckman(
        endog=y_out,
        exog=X_out,
        selection=D,
        exog_selection=X_sel
    ).fit()

    heckman_result = heckman
    heckman_fitted = True

    print("\nHeckman estimation successful.")

    # Display key attributes
    if hasattr(heckman, 'params'):
        print(f"\nOutcome equation parameters: {heckman.params}")
    if hasattr(heckman, 'imr_coef'):
        print(f"IMR coefficient (delta = rho*sigma_eps): {heckman.imr_coef:.4f}")
    if hasattr(heckman, 'rho'):
        print(f"Selection correlation (rho):              {heckman.rho:.4f}")

    if hasattr(heckman, 'summary'):
        try:
            print("\n" + str(heckman.summary()))
        except Exception as se:
            print(f"(summary() not available: {se})")

except ImportError as ie:
    print(f"PanelHeckman import failed: {ie}")
    print("Proceeding with manual two-step Heckman for demonstration.")

except Exception as e:
    print(f"Heckman estimation failed: {e}")
    print("Proceeding with manual two-step Heckman for demonstration.")

In [ ]:
# Manual two-step Heckman (always runs as fallback or demonstration)
print("=" * 60)
print("Manual Two-Step Heckman Estimator")
print("(for pedagogical clarity and as fallback)")
print("=" * 60)

try:
    df_h = load_dataset('mroz')
    df_h['log_wage'] = np.where(df_h['wage'] > 0,
                                 np.log(df_h['wage'].clip(lower=0.01)),
                                 np.nan)

    sel_vars  = ['educ', 'age', 'kidslt6', 'kidsge6', 'nwifeinc']
    out_vars  = ['educ', 'exper', 'expersq']
    D_h       = df_h['inlf'].values

    # ---- Step 1: Probit on full sample -----------------------------------
    from scipy.optimize import minimize
    from scipy.stats import norm as norm_dist

    X_sel_h = np.column_stack([np.ones(len(df_h))] + [df_h[v].values for v in sel_vars])
    sel_names = ['const'] + sel_vars

    def neg_ll_probit(gamma):
        xg = X_sel_h @ gamma
        ll = D_h * norm_dist.logcdf(xg) + (1 - D_h) * norm_dist.logcdf(-xg)
        return -ll.sum()

    gamma0 = np.zeros(X_sel_h.shape[1])
    res_probit = minimize(neg_ll_probit, gamma0, method='BFGS',
                          options={'maxiter': 2000, 'disp': False})
    gamma_hat = res_probit.x

    # ---- Step 2: Compute IMR and OLS on selected sample -----------------
    xg_hat = X_sel_h @ gamma_hat
    phi_xg  = norm_dist.pdf(xg_hat)
    Phi_xg  = norm_dist.cdf(xg_hat)
    imr     = np.where(Phi_xg > 1e-10, phi_xg / Phi_xg, -xg_hat)

    # Keep only selected observations (D=1)
    sel_mask = D_h == 1
    df_sel = df_h[sel_mask].copy()
    imr_sel = imr[sel_mask]

    X_out_h = np.column_stack(
        [np.ones(sel_mask.sum())] + [df_sel[v].values for v in out_vars] + [imr_sel]
    )
    out_names = ['const'] + out_vars + ['IMR']
    y_sel = df_sel['log_wage'].values

    # OLS: beta = (X'X)^{-1} X'y
    beta_heck = np.linalg.lstsq(X_out_h, y_sel, rcond=None)[0]
    fitted_h  = X_out_h @ beta_heck
    resid_h   = y_sel - fitted_h
    sigma_h   = np.std(resid_h)
    imr_coef  = beta_heck[-1]  # coefficient on IMR = delta = rho * sigma_eps

    print("\nStep 1 — Probit (selection equation):")
    for i, name in enumerate(sel_names):
        print(f"  {name:<15} gamma = {gamma_hat[i]:>8.4f}")

    print(f"\nStep 2 — OLS on selected sample (N={sel_mask.sum()}):")
    for i, name in enumerate(out_names):
        print(f"  {name:<15} beta  = {beta_heck[i]:>8.4f}")

    print(f"\nIMR coefficient (delta = rho * sigma):  {imr_coef:.4f}")
    rho_approx = imr_coef / sigma_h if sigma_h > 0 else np.nan
    print(f"Approx. selection correlation (rho):    {rho_approx:.4f}")

    if abs(imr_coef) > 0.05:
        print("\nIMR coef is non-negligible → selection bias present → Heckman preferred.")
    else:
        print("\nIMR coef is close to zero → weak selection → OLS may be adequate.")

    heckman_manual_ok = True

    # Store for ME decomposition
    heckman_gamma   = gamma_hat
    heckman_beta    = beta_heck[:-1]   # excluding IMR
    heckman_delta   = imr_coef
    heckman_out_names = ['const'] + out_vars
    heckman_sel_names = ['const'] + sel_vars
    X_sel_full       = X_sel_h
    xg_full          = xg_hat

except Exception as e:
    print(f"Manual Heckman failed: {e}")
    heckman_manual_ok = False

In [ ]:
# Heckman ME decomposition: Direct + Indirect
if heckman_manual_ok:
    print("=" * 60)
    print("Heckman ME Decomposition")
    print("Direct:   partial E[Y|D=1,X]/partial x_k = beta_k")
    print("Indirect: delta * partial lambda(Zg)/partial x_k")
    print("Total:    Direct + Indirect")
    print("=" * 60)

    # d/dz [lambda(z)] = -lambda(z)*(z + lambda(z))
    phi_xg_full  = norm.pdf(xg_full)
    Phi_xg_full  = norm.cdf(xg_full)
    lambda_full  = np.where(Phi_xg_full > 1e-10,
                            phi_xg_full / Phi_xg_full,
                            -xg_full)
    dlambda_dz   = -lambda_full * (xg_full + lambda_full)  # d lambda/dz

    # Variables that appear in BOTH selection and outcome
    common_vars = [v for v in out_vars if v in sel_vars]
    outcome_only = [v for v in out_vars if v not in sel_vars]

    rows_heck = []
    for v in out_vars:
        out_idx = heckman_out_names.index(v) if v in heckman_out_names else None
        sel_idx = heckman_sel_names.index(v) if v in heckman_sel_names else None

        direct = heckman_beta[out_idx] if out_idx is not None else 0.0

        if sel_idx is not None:
            # AME version of indirect effect
            gamma_k = heckman_gamma[sel_idx]
            indirect = heckman_delta * np.mean(gamma_k * dlambda_dz)
        else:
            indirect = 0.0  # variable not in selection equation

        rows_heck.append({
            'Variable': v,
            'Direct ME (beta_k)': direct,
            'Indirect ME (selection)': indirect,
            'Total ME': direct + indirect,
            'Note': 'Both eqs' if sel_idx is not None else 'Outcome only'
        })

    decomp_heck = pd.DataFrame(rows_heck).set_index('Variable')
    print()
    print(decomp_heck.to_string())
    print()
    print("Note: Indirect ME is non-zero only for variables that appear")
    print("      in the selection equation.")

    heckman_decomp_ok = True
    heckman_rows = rows_heck

else:
    print("Heckman decomposition not available.")
    heckman_decomp_ok = False

In [ ]:
# Stacked bar: Direct + Indirect = Total
if heckman_decomp_ok:
    vars_h     = [r['Variable'] for r in heckman_rows]
    direct_v   = [r['Direct ME (beta_k)']        for r in heckman_rows]
    indirect_v = [r['Indirect ME (selection)']    for r in heckman_rows]

    fig, ax = plt.subplots(figsize=(8, 5))
    x = np.arange(len(vars_h))

    ax.bar(x, direct_v, label='Direct Effect ($\\beta_k$)',
           color='steelblue', alpha=0.85)
    ax.bar(x, indirect_v, bottom=direct_v,
           label='Indirect / Selection Effect ($\\delta \\cdot \\partial\\lambda/\\partial x$)',
           color='sandybrown', alpha=0.85)

    # Total label
    for i, r in enumerate(heckman_rows):
        total_val = r['Total ME']
        ax.text(i, total_val + 0.002,
                f'{total_val:.3f}', ha='center', va='bottom', fontsize=8, fontweight='bold')

    ax.axhline(0, color='black', lw=0.8)
    ax.set_xticks(x)
    ax.set_xticklabels(vars_h, rotation=15, ha='right')
    ax.set_ylabel("Marginal Effect on log(Wage)")
    ax.set_title(
        "Heckman ME Decomposition: Direct vs Selection Effect\n"
        "Wages conditional on LFP=1 — Mroz (1987)"
    )
    ax.legend(fontsize=9)
    plt.tight_layout()
    plt.savefig('../outputs/plots/04_heckman_decomposition.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Plot saved.")
else:
    print("Skipping Heckman plot — decomposition not available.")

---

<a id='section4'></a>
## Section 4: When to Use Tobit vs Heckman?

Both models address limited dependent variables, but the underlying economic situations differ:

| Situation | Model | Key Assumption |
|-----------|-------|----------------|
| **Corner solution**: person actively chooses $Y=0$ (e.g., zero charitable giving; does not want to give) | **Tobit** | $Y^*$ (latent desired amount) exists for *all* individuals |
| **Sample selection**: $Y$ is unobserved for non-participants (e.g., wage unobserved for non-workers) | **Heckman** | Selection process is *separate* from outcome process |

### Decision rule

> Ask: "Does a person with $Y=0$ have a 'desired $Y$'?"
> - **Yes** (they chose zero) $\Rightarrow$ **Tobit**
> - **No** (they are simply excluded from observation) $\Rightarrow$ **Heckman**

### Empirical test

If the IMR coefficient $\delta$ is **significantly different from zero** $\Rightarrow$ selection bias is present; use Heckman.  
If $\delta \approx 0$ $\Rightarrow$ no selection bias; Tobit or OLS may suffice.

In [ ]:
print("=" * 60)
print("TOBIT vs HECKMAN: Summary Comparison")
print("=" * 60)

if tobit_fitted:
    print(f"\nTobit (Corner Solution):")
    print(f"  Example dataset: Mroz hours worked")
    print(f"  % censored at zero: {(df['hours'] == 0).mean():.1%}")
    print(f"  Sigma (latent error): {tobit.sigma:.2f}")

    if mcdonald_ok:
        educ_ext = float(extensive_series['educ'])
        educ_int = float(intensive_series['educ'])
        educ_tot = float(total_series['educ'])
        if abs(educ_tot) > 1e-6:
            print(f"\n  McDonald-Moffitt for 'educ':")
            print(f"    Extensive margin: {educ_ext:.2f} "
                  f"({100*educ_ext/educ_tot:.0f}% of total)")
            print(f"    Intensive margin: {educ_int:.2f} "
                  f"({100*educ_int/educ_tot:.0f}% of total)")
            print(f"    Total:            {educ_tot:.2f}")

if heckman_manual_ok:
    print(f"\nHeckman (Sample Selection):")
    print(f"  Example dataset: Mroz wages (observed only for LFP=1)")
    print(f"  IMR coefficient (delta): {heckman_delta:.4f}")
    print(f"  Approx. rho:             {rho_approx:.4f}")

    # IMR significance test
    print(f"\n  IMR significance test:")
    resid_std = np.std(y_sel - X_out_h @ beta_heck)

    # Approximate SE for IMR coefficient via OLS vcov
    try:
        XtX_inv = np.linalg.inv(X_out_h.T @ X_out_h)
        se_full  = np.sqrt(np.diag(XtX_inv) * resid_std ** 2)
        se_imr   = se_full[-1]  # SE for IMR coefficient
        t_imr    = heckman_delta / se_imr
        p_imr    = 2 * (1 - norm.cdf(abs(t_imr)))
        print(f"    delta = {heckman_delta:.4f}")
        print(f"    SE    = {se_imr:.4f}")
        print(f"    t     = {t_imr:.2f}")
        print(f"    p     = {p_imr:.4f}")
        if p_imr < 0.05:
            print(f"    --> Selection bias is statistically significant (p<0.05).")
            print(f"        Heckman correction recommended over OLS.")
        else:
            print(f"    --> Selection bias is not significant (p>0.05).")
            print(f"        OLS may be adequate here.")
    except Exception as e:
        print(f"    (Cannot compute SE: {e})")

---

<a id='takeaways'></a>
## Key Takeaways

1. **Tobit Type 1** (latent ME = $\beta$): answers the question "what would happen if censoring were lifted?"

2. **Tobit Type 2** (unconditional = $\beta \cdot \Phi(z)$): the population-average effect including
   all observations — the right measure for **policy analysis**.

3. **Tobit Type 3** (conditional = $\beta \cdot [1 - \lambda(z+\lambda)]$): effect for non-censored
   observations — use for **intensive-margin analysis**.

4. **McDonald-Moffitt decomposition**: unconditional ME = extensive margin + intensive margin.
   Reveals *where* a variable's effect operates (participation decision vs amount decision).

5. **Heckman ME**: total = direct ($\beta_k$) + indirect (selection through IMR).  
   The indirect component is **non-zero only when** the covariate also appears in the selection equation.

6. **Tobit vs Heckman**: choose based on **economic theory**, not just statistics:
   - Corner solution (agent actively chooses zero) → **Tobit**  
   - Sample selection (outcome not observed for non-participants) → **Heckman**
   - Test: IMR significance ($\delta \neq 0$) supports Heckman.

---

**Next notebook (05):** Interaction effects in nonlinear models require computing cross-partial derivatives.
A common error is to simply interpret the interaction coefficient — this is incorrect for probit, logit,
Tobit and other nonlinear models.

In [ ]:
# Export results
import os
os.makedirs('../outputs/tables', exist_ok=True)

saved = []

# Tobit AME tables
if tobit_fitted and ame_uncond_ok:
    try:
        tbl = format_me_table(ame_uncond)
        tbl.to_csv('../outputs/tables/04_tobit_ame_unconditional.csv', index=False)
        saved.append('04_tobit_ame_unconditional.csv')
    except Exception as e:
        print(f"Could not save unconditional AME table: {e}")

if tobit_fitted and ame_cond_ok:
    try:
        tbl = format_me_table(ame_cond)
        tbl.to_csv('../outputs/tables/04_tobit_ame_conditional.csv', index=False)
        saved.append('04_tobit_ame_conditional.csv')
    except Exception as e:
        print(f"Could not save conditional AME table: {e}")

# McDonald-Moffitt decomposition
if mcdonald_ok:
    try:
        decomp.to_csv('../outputs/tables/04_mcdonald_moffitt.csv')
        saved.append('04_mcdonald_moffitt.csv')
    except Exception as e:
        print(f"Could not save McDonald-Moffitt table: {e}")

# Heckman decomposition
if heckman_decomp_ok:
    try:
        decomp_heck.to_csv('../outputs/tables/04_heckman_decomposition.csv')
        saved.append('04_heckman_decomposition.csv')
    except Exception as e:
        print(f"Could not save Heckman decomposition: {e}")

if saved:
    print("Files saved to ../outputs/tables/:")
    for f in saved:
        print(f"  - {f}")
else:
    print("No tables saved (models may not have converged).")

print("\nNotebook 04 complete.")